# Erläuterung Paper

 ## Prämisse des Papers

## Aufbau des Papers und des Modells

## Inspiration für das durchgeführte Projekt

In dem unserem Projekt zugrunde liegenden Paper werden die Rohdaten hochauflösender Radarsensoren in ein neuronales Netz gespeist. Dieses Netz soll den sogenannten "Free Driving Space", also den hindernisfreien Bereich der Fahrbahn vor dem Fahrzeug, berechnen. Die Herausforderung darin besteht nicht nur in der Komplexität des Netzes, sondern auch in der dafür notwendigen enormen Rechenleistung, um schnell die richtigen Ergebnisse zu erhalten.

Die Motivation, dieses Projekt durchzuführen folgte aus der Frage: Ist es möglich ein Neuronales Netz gegen hochverarbeitete Daten zu trainieren? Und falls ja, kann man damit qualitativ ähnliche Ergebnisse zu erzielen? 

Dies herangehensweise würde die erforderliche Rechenleistung stark reduzieren und eine Anwendung im Fahrzeug erleichtern.

# Grundlagenwissen Radar

## Radarpunktwolken

# Zugrundeliegende Daten

## Datensatz

## Label

# Modell

### Anmerkungen
Der Code zu diesem Projekt ist unter folgenem Link auf unserem GitHub Repository zu finden: https://github.com/zaph01/ImageRadar
